# Data Prepareation:

## Data Cleaning

### Function for hidden code cells

### Read all the files 

Change file path to path of your raw data folder (both search & purchase data)

In [33]:
#@title
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
#@title
# Import Libraries
import pandas as pd
import csv 
import numpy as np
import glob
import matplotlib.pyplot as plot
import plotly as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

#Path of the folder
file_path = r'/content/drive/MyDrive/Colab Notebooks/Data Files/SJCOG Vamos' 

#Find all csv files in this folder
all_files = sorted(glob.glob(file_path + "/*.csv"))

#Create a empty list
files = []

for filename in all_files:
    
    #Read each file in that folder
    df = pd.read_csv(filename, index_col=None, header=0)
    
    #Append them to the empty list
    files.append(df)

### Preview

In [35]:
#@title
#Let's look at first 5 rows of our first file
files[0].head(5)

,CreatedAt,UserIDPseudonymized,OrderId,KyytiServiceNames,KyytiServiceIDs,KyytiProductNames,KyytiProductIDs,KyytiProductPrices,Price,Currency,PaymentMethodType
0,2020-03-04 16:07:59,GDNRV3x6EuQx32o9f1U8dFzSCG0=,1ba2abe7-8507-4408-8c22-443799712fec,VoGo California,6c091afa-0728-4150-9241-d851077b40ca,VoGO,56693cb4-544d-40fd-af14-6b50b0a90c24,0,0.0,USD,TUUP_FREE
1,2020-03-05 08:51:29,bvZ0sww0ITOf9M9mA/cD6RgbVhA=,2b8e46df-0fca-438a-ac02-f5059c0fd7cc,VoGo California,6c091afa-0728-4150-9241-d851077b40ca,VoGO,e1800d13-51a9-4e94-a28e-2e25148b7c27,0,0.0,USD,TUUP_CREDIT
2,2020-03-17 15:25:55,GDNRV3x6EuQx32o9f1U8dFzSCG0=,72179031-9ab1-40d3-89aa-6783ebd31403,VoGo California,6c091afa-0728-4150-9241-d851077b40ca,VoGO,3ec0b060-73d8-416a-b995-b203a8fc21cd,0,0.0,USD,TUUP_FREE
3,2020-03-18 13:27:06,bvZ0sww0ITOf9M9mA/cD6RgbVhA=,fc10688c-fda7-40c9-a80a-fa33800042ad,VoGo California,6c091afa-0728-4150-9241-d851077b40ca,VoGO,8234d208-4be3-4d6d-a7bb-2a68b15c0d4c,0,0.0,USD,TUUP_FREE
4,2020-03-18 13:44:31,5lPv7YNmTD5nsHdIMqFGlUGHoaI=,44fa8d59-e922-492c-81ac-d6bf04e49743,VoGo California,6c091afa-0728-4150-9241-d851077b40ca,VoGO,960517d0-1cd4-4301-9819-6f0a50a0c4bd,0,0.0,USD,TUUP_CREDIT


### Show all the column names of the files

In [36]:
#@title
for i in range(len(files)):
    
    #Find column names of each file
    a=files[i].columns
    
    #Print them out
    print(a)

Index(['CreatedAt', 'UserIDPseudonymized', 'OrderId', 'KyytiServiceNames',
       'KyytiServiceIDs', 'KyytiProductNames', 'KyytiProductIDs',
       'KyytiProductPrices', 'Price', 'Currency', 'PaymentMethodType'],
      dtype='object')
Index(['CreatedAt', 'UserID', 'OrderId', 'KyytiServiceNames',
       'KyytiServiceIDs', 'KyytiProductNames', 'KyytiProductIDs',
       'KyytiProductPrices', 'Price', 'Currency', 'PaymentMethodType'],
      dtype='object')
Index(['CreatedAt', 'UserID', 'OrderId', 'KyytiServiceNames',
       'KyytiServiceIDs', 'KyytiProductNames', 'KyytiProductIDs',
       'KyytiProductPrices', 'Price', 'Currency', 'PaymentMethodType'],
      dtype='object')
Index(['CreatedAt', 'UserID', 'OrderId', 'KyytiServiceNames',
       'KyytiServiceIDs', 'KyytiProductNames', 'KyytiProductIDs',
       'KyytiProductPrices', 'Price', 'Currency', 'PaymentMethodType'],
      dtype='object')
Index(['RequestTime', 'UserIDPseudonymized', 'RoutingID', 'SortingIndex',
       'MainMode', 'Trave

There are several files' user ID column is named as 'UserIDPseudonymized' so we need to change it to 'UserID' before we merge them.

In [37]:
#@title
for i in range(len(files)):

    # If the second column name of the file is 'UserIDPseudonymized'
    if files[i].columns[1] == 'UserIDPseudonymized':

        # Change it to UserID
        files[i] = files[i].rename(columns={'UserIDPseudonymized': 'UserID'})

### Combine all purchase files

In [38]:
#@title
# Set first file as purchase
purchase = files[0]

# Create a empty list
cc = list()
for i in range(1, len(files)):

    # a is all the column names of purchase
    a = purchase.columns

    # b is all the column names of the file we detect
    b = files[i].columns

    # If all of there column names are the same
    if a.all() == b.all():

        # Conbine them with together
        purchase = pd.concat([purchase, files[i]])
    else:

        # Otherwise, append the number of i into the empty list cc
        cc.append(i)
        pass

In [39]:
#@title
# Sort the purchase data based on "CreatedAt" column
purchase = purchase.sort_values(by="CreatedAt")

# Show the first 5 rows of purchase data
purchase.head(5)

,CreatedAt,UserID,OrderId,KyytiServiceNames,KyytiServiceIDs,KyytiProductNames,KyytiProductIDs,KyytiProductPrices,Price,Currency,PaymentMethodType
0,2020-03-04 16:07:59,GDNRV3x6EuQx32o9f1U8dFzSCG0=,1ba2abe7-8507-4408-8c22-443799712fec,VoGo California,6c091afa-0728-4150-9241-d851077b40ca,VoGO,56693cb4-544d-40fd-af14-6b50b0a90c24,0,0.0,USD,TUUP_FREE
1,2020-03-05 08:51:29,bvZ0sww0ITOf9M9mA/cD6RgbVhA=,2b8e46df-0fca-438a-ac02-f5059c0fd7cc,VoGo California,6c091afa-0728-4150-9241-d851077b40ca,VoGO,e1800d13-51a9-4e94-a28e-2e25148b7c27,0,0.0,USD,TUUP_CREDIT
2,2020-03-17 15:25:55,GDNRV3x6EuQx32o9f1U8dFzSCG0=,72179031-9ab1-40d3-89aa-6783ebd31403,VoGo California,6c091afa-0728-4150-9241-d851077b40ca,VoGO,3ec0b060-73d8-416a-b995-b203a8fc21cd,0,0.0,USD,TUUP_FREE
3,2020-03-18 13:27:06,bvZ0sww0ITOf9M9mA/cD6RgbVhA=,fc10688c-fda7-40c9-a80a-fa33800042ad,VoGo California,6c091afa-0728-4150-9241-d851077b40ca,VoGO,8234d208-4be3-4d6d-a7bb-2a68b15c0d4c,0,0.0,USD,TUUP_FREE
4,2020-03-18 13:44:31,5lPv7YNmTD5nsHdIMqFGlUGHoaI=,44fa8d59-e922-492c-81ac-d6bf04e49743,VoGo California,6c091afa-0728-4150-9241-d851077b40ca,VoGO,960517d0-1cd4-4301-9819-6f0a50a0c4bd,0,0.0,USD,TUUP_CREDIT


### Combine all search files

In [40]:
#@title
#Set the first file that is different than the previous files as search
search=files[cc[0]]


for i in range(cc[0]+1,len(files)):
    
    #a is all the column names of search
    a=search.columns
    
    #b is all the column names of the file we detect
    b=files[i].columns
    
    #If all of there column names are the same
    if a.all()==b.all():
        
        #Conbine them with together
        search=pd.concat([search, files[i]])
    else:
        
        #Otherwise, pass this file
        pass
    

### Drop column: 'LineShortNames', 'KyytiServiceNames','KyytiServiceIDs', and 'SourceName' from search file

In [41]:
#@title
#Drop those columns from search data
search.drop(['LineShortNames', 'KyytiServiceNames','KyytiServiceIDs','SourceName'], axis=1, inplace=True)

#Sort the search data based on "RequestTime" column
search= search.sort_values(by="RequestTime")

#Show the first 5 rows of search data
search.head(5)

,RequestTime,UserID,RoutingID,SortingIndex,MainMode,TravelModes,AgencyNames,LineLongNames,DepartureTime,MinimumTravelTimeSeconds,MaximumTravelTimeSeconds,StartBlockGeoID,EndBlockGeoID,SelectedCount,LastSelected,Purchased,PurchaseId
0,2020-01-13 15:31:07,x5bGAQ8vLBiA8xqRdX2DET7rBCo=,7e60305a-6016-4e1d-854a-c756e8cbbf3a,0,bicycle,bicycle,NaN,NaN,2020-01-13 15:31:07,2627,2627,60770038033042,60770051242000,0,False,False,NaN
1,2020-01-13 15:31:41,x5bGAQ8vLBiA8xqRdX2DET7rBCo=,83444594-395f-4ebf-8ddb-10e605a4c3a7,0,bicycle,bicycle,NaN,NaN,2020-01-13 15:31:41,77,77,60770051242004,60770051242000,0,False,False,NaN
2,2020-01-13 23:28:03,FsXK6bGQXavEISIKy4wD8w9UM5k=,31a0a996-b245-455e-a0c9-352dece09022,0,publicTransport,"walk,bus,walk,bus,walk,rail,walk",NaN,"Route 31,Ace Express,Altamont Corridor Express",2020-01-13 23:51:36,11755,11755,60990005012041,60770001002008,2,False,False,NaN
3,2020-01-13 23:28:03,FsXK6bGQXavEISIKy4wD8w9UM5k=,31a0a996-b245-455e-a0c9-352dece09022,1,publicTransport,"walk,bus,walk,bus,walk,rail,walk",NaN,"Route 28,Ace Express,Altamont Corridor Express",2020-01-14 00:27:00,13231,13231,60990005012041,60770001002008,1,True,False,NaN
4,2020-01-13 23:28:03,FsXK6bGQXavEISIKy4wD8w9UM5k=,31a0a996-b245-455e-a0c9-352dece09022,2,publicTransport,"walk,bus,walk,bus,walk,rail,walk",NaN,"Route 31,Ace Express,Altamont Corridor Express",2020-01-14 01:49:05,12086,12086,60990005012041,60770001002008,0,False,False,NaN


## Data Shape

In [42]:
#@title
#Print the shape of pruchase data
print('There are',purchase.shape[0],'rows and ',purchase.shape[1],'columns in purchase data')

#Print the shape of search data
print('There are',search.shape[0],'rows and ',search.shape[1],'columns in search data')

There are 574 rows and  11 columns in purchase data
There are 11153 rows and  17 columns in search data


## Missing Values

In [43]:
#@title
#Show the number of missing values in each column
purchase.isnull().sum()

CreatedAt              0
UserID                 0
OrderId                0
KyytiServiceNames      0
KyytiServiceIDs        0
KyytiProductNames      0
KyytiProductIDs        0
KyytiProductPrices     0
Price                  0
Currency               0
PaymentMethodType     19
dtype: int64

In [44]:
#@title
#Show the number of missing values in each column
search.isnull().sum()

RequestTime                     0
UserID                          0
RoutingID                       0
SortingIndex                    0
MainMode                        0
TravelModes                     0
AgencyNames                  2277
LineLongNames                1953
DepartureTime                   0
MinimumTravelTimeSeconds        0
MaximumTravelTimeSeconds        0
StartBlockGeoID                 0
EndBlockGeoID                   0
SelectedCount                   0
LastSelected                    0
Purchased                       0
PurchaseId                  11125
dtype: int64

**We could see that there are 19 missing values in purchase data and there are three columns that have missing values in search data.**

# Functions Code

## Combine

In [45]:
def combine(*file_name): # Take a list of file names，tuple (order is insignificant)
    
    file_name = list(file_name) # Convert tuple into list (could be shorten)
    for i in range(len(file_name)): # convert xlsx into csv if detected 
        if file_name[i][-4:] == 'xlsx':
            pd.read_excel(file_name[i]).to_csv(file_name[i][:-4] + 'csv',index = False) # add sheet option in the future if needed
            file_name[i] = file_name[i][:-4] + 'csv'
    file_name = tuple(file_name) # Convert list into tuple(could be shorten)        
    
    flag = 1             # Initial the status (flag=1 indicats move on)
    current_columns = pd.read_csv(file_name[0]).columns  # extract name of first column
    for i in range(len(file_name)-1):                   # compare with each in all files
        next_columns = pd.read_csv(file_name[i+1]).columns
        if (all(current_columns) == all(next_columns)):
            current_columns = next_columns
        else:
            print ('Error!')
            print ('File'+file_name[i]+'and File'+file_name[i+1]+'have different features.')
            flag = 0     # Shut the program if columns names do not match and warn user
            break
            
    if (flag):          # Move on if all files matches
        df = pd.DataFrame() 
        for file in file_name: # merge all files
            new_df = pd.read_csv(file)
            df = pd.concat([df,new_df],axis=0)
            
        df[df.columns[0]] = pd.to_datetime(df[df.columns[0]]) # change first column as timestamped
        df = df.sort_values(by=df.columns[0]) # sort the data as time ascending order
        # df.drop("no needed columns")
        return df

## Count

### Count by period:

In [46]:
def count_by_period(dates,period): #dates is a column of timestamp，period can be (day,month, quarter)
    period = str.lower(period)    # Ignore case of input
    if (period == 'day'): 
        dates = dates.dt.floor('d').value_counts()
        return dates
    elif(period == 'month'):
        dates = dates.dt.to_period('M').value_counts()
        return dates
    elif(period == 'quarter'):
        dates = dates.dt.to_period('Q').value_counts()
        return dates
    elif (period == 'hour'):
        dates = dates.dt.to_period('H').value_counts()
        return dates
    else:
        print('Please input day,month,or quarter for the second argument.')

### Count search:

In [47]:
def count_use(df,YEAR,period):       
    if type(YEAR) == str:
        YEAR = int(YEAR)
    df_copy = df[df[df.columns[0]].dt.year == YEAR].copy() 
    key = list(zip(df_copy[df_copy.columns[0]],df_copy[df_copy.columns[1]])) 
    key = set(key)              
    df_key = pd.DataFrame(key) 
    df_key = df_key.sort_values(by=df_key.columns[0]) 
    counts = pd.DataFrame(count_by_period(df_key[0],period)) 
    counts = counts.sort_index() 
    return counts

In [48]:
def count_use1(df,period):       # period will be pass to count_by_period()
    key = list(zip(df[df.columns[0]],df[df.columns[1]])) # # Combine date and userID as the key column
    key = set(key)              #  # Use set to remove the duplicated records
    df_key = pd.DataFrame(key)  
    df_key = df_key.sort_values(by=df_key.columns[0]) # Sort by date
    counts = pd.DataFrame(count_by_period(df_key[0],period)) # Count occurence of dates
    counts = counts.sort_index() # Sort by date
    return counts

## Plot

### Plot by period:

In [49]:
def plot_by_period(fileName,period):
    period = period.lower()
    file = combine(fileName)
    periodic = count_use(file,period)
    if (period=='day'):
        daily = periodic
        trace = go.Scatter(x = list(daily.index),y=daily[0])  # x is date, y is counts
        fig = go.Figure(trace)
        fig.update_layout(title = 'Daily Counts',
                 xaxis_title = 'Date',
                 yaxis_title = 'Counts(times)')
    elif (period=='month'):
        monthly = periodic
        monthly.index = monthly.index.strftime('%Y-%m') # Convert PeriodIndex to normal Index
        trace = go.Scatter(x = monthly.index.to_series(),y=monthly[0])
        fig = go.Figure(trace)
        fig.update_layout(title = 'Monthly Counts',
                          xaxis_title = 'Date',
                          yaxis_title = 'Counts(times)')
    elif(period == 'quarter'):
        quarterly = periodic
        trace = go.Scatter(x = (quarterly.index.to_series().astype(str)),y=list(quarterly[0]))
        fig = go.Figure(trace)
        fig.update_layout(title = 'Quarterly Counts',
                         xaxis_title = 'Quarter',
                         yaxis_title = 'Counts(times)')
    fig.show()

# Final Files

## Export search & purchase final file

In [50]:
#Write out those final file as csv
search.to_csv('search.csv', index=False)
purchase.to_csv('purchase.csv', index=False)

## Sort final files by time

In [51]:
search['RequestTime'] = pd.to_datetime(search.RequestTime,
                                       format='%Y-%m-%d %H:%M:%S')
purchase['CreatedAt'] = pd.to_datetime(purchase.CreatedAt,
                                       format='%Y-%m-%d %H:%M:%S')

#Sort their value by time
search = search.sort_values(by="RequestTime")
purchase = purchase.sort_values(by="CreatedAt")

# Visualization

## Daily Usage by Year

In [52]:
#@title
daily_2021 = count_use(search,'2021','day')   # period 输入 day 或Day 或DAY都是按日统计，默认把日期当成了index
trace = go.Scatter(x = list(daily_2021.index),y=daily_2021[0])  # 把日期（index）赋值给x, 计数counts 赋值给Y
fig = go.Figure(trace)
fig.update_layout(title = '2021 Daily Usage',
                 xaxis_title = 'Date',
                 yaxis_title = 'Usage(times)')
fig.show()

In [53]:
#@title
daily_2020 = count_use(search,'2020','day')   
trace = go.Scatter(x = list(daily_2020.index),y=daily_2020[0])  
fig = go.Figure(trace)
fig.update_layout(title = '2020 Daily Usage',
                 xaxis_title = 'Date',
                 yaxis_title = 'Usage(times)')
fig.show()

## Montly Usage Comparision Between Years

In [54]:
#@title
monthly_2021 = count_use(search,'2021','month')   # period 输入 day 或Day 或DAY都是按日统计，默认把日期当成了index
monthly_2020 = count_use(search,'2020','month')

**This is the comparison of monthly usage amount between 2020 and 2021**

In [55]:
#@title
fig = go.Figure()
fig.add_trace(go.Scatter(x = list(monthly_2021.index.strftime("%m")),y=monthly_2021[0],name = '2021'))
fig.add_trace(go.Scatter(x = list(monthly_2020.index.strftime("%m")),y=monthly_2020[0],name = '2020'))
fig.update_layout(title = 'Monthly Usage',
                 xaxis_title = 'Month',
                 yaxis_title = 'Usage(times)')
fig.show()

## Quarterly Usage Compare Between Years

In [56]:
#@title
quarterly_2021 = [sum(monthly_2021[0:3][0]),sum(monthly_2021[3:6][0]),sum(monthly_2021[6:9][0]),sum(monthly_2021[9:12][0])]  # 根据月份的数据手动算季度数据
quarterly_2020 = [sum(monthly_2020[0:3][0]),sum(monthly_2020[3:6][0]),sum(monthly_2020[6:9][0]),sum(monthly_2020[9:12][0])]

**This is the comparison of quarterly usage amount between 2020 and 2021**

In [57]:
#@title
fig = go.Figure()

fig.add_trace(go.Scatter(x = ['Q1','Q2','Q3','Q4'],y = quarterly_2021,name = '2021'))
fig.add_trace(go.Scatter(x = ['Q1','Q2','Q3','Q4'],y = quarterly_2020,name = '2020'))

fig.update_layout(title = 'Quarterly Usage',
                 xaxis_title = 'Quarter',
                 yaxis_title = 'Usage(Total Times)')
fig.show()

## Compare Total Amount and Quantity of ticket purchases

In [58]:
#@title
monthly = count_use(purchase,2021,'month')

In [59]:
#@title
def count_ticket(df,YEAR):       # df 是总文件，period 待会pass给count_by_period()
    if type(YEAR) == str:
        YEAR = int(YEAR)
    df_copy = df[df[df.columns[0]].dt.year == YEAR].copy() # 筛选固定的年
    grouped = pd.DataFrame(zip(df_copy['CreatedAt'].dt.month,df_copy['Price']))
    total = grouped.groupby(by = grouped.columns[0]).sum()
    return total

In [60]:
#@title
total = count_ticket(purchase, 2021)

**This graph depicts the number of tickets sold each month in 2021, as well as the total amount from those sales.**

In [61]:
#@title
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x = list(total.index),y=total[1], name = '$ Amount of Sale'))
fig.add_trace(go.Scatter(x = list(monthly.index.strftime("%m")),y=monthly[0], name = 'Number of Sale'),secondary_y=True)
fig.update_layout(title = 'Monthly Ticket Sale in USD 2021',
                 xaxis_title = 'Month',
                 yaxis_title = 'USD')
fig.update_yaxes(title_text="Number of Ticket Sale", secondary_y=True)
fig.show()

## Agency Proportion by Queried + Selected 

**For the route that finally been selected, we counted and summarized the proportion of agency. If there is more than one agency in that route then we will take them all in the count. According to the result, we could see that San Joaquin Regional District takes the most part of the proportion which is about 40.3 %.** 

In [62]:
#@title
agency_select_count = search[search['LastSelected'] == True][['AgencyNames','LastSelected']]#.groupby(by = ['AgencyNames']).sum()
agency_select_count = pd.DataFrame(agency_select_count['AgencyNames'].str.split(',').explode())
agency_select_count['LastSelected'] = True
agency_select_count = agency_select_count.groupby(by = ['AgencyNames']).sum().sort_values(by = ['LastSelected'], ascending = False)
fig = px.pie(agency_select_count, values='LastSelected', names=agency_select_count.index, title='2020 -2021 Agency Proportion (Queried and Selected)')
fig.show()

## 2020-2021 Hour Usage

**The histogram shows the usage amount of each hour from 2020 to 2021**

In [63]:
#@title
# Hour Usage Code
aaa = count_use1(search, 'hour')

aaa.reset_index(inplace=True)
aaa = aaa.rename(columns={'index': 'time'})
aaa.columns = ['time', 'count']

aaa["time"] = aaa["time"].astype(str).str[10:]
df2 = aaa.groupby(['time'], as_index=False).sum()

trace0 = go.Bar(x=df2.time, y=df2["count"])

fig = go.Figure(trace0)
fig.update_layout(title='2020-2021 Hour Usage',
                  xaxis_title='Hour',
                  yaxis_title='Counts(times)')
fig.show()